In [38]:
import requests as req
import pandas as pd
import sqlite3

In [39]:
date = '20221109'
url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX'
payloads = {
  'response': 'html',
  'date': date,
  'type': 'ALLBUT0999'
}
headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.5249.119 Safari/537.36'
}

In [40]:
res = req.get(url, params= payloads, headers=headers)

In [41]:
try:
  df = pd.read_html(res.text)[-1]
except:
  df = None

df.columns = df.columns.get_level_values(2)
df.drop(['證券名稱','漲跌(+/-)'], axis=1, inplace=True)

df['日期'] = pd.to_datetime(date)
df = df.set_index(['證券代號', '日期'])

df = df.apply(pd.to_numeric, errors='coerce')
df.drop(df[df['收盤價'].isnull()].index, inplace=True)

In [42]:
def save_daily_prices(new_df):

  connection = sqlite3.connect('data.db')
  try:
    df = pd.read_sql('select * from daily_prices', connection, index_col=['證券代號', '日期'], parse_dates=['日期'])
  except:
    df = pd.DataFrame()
  combined_df = df.append(new_df, sort=False)
  final_df = combined_df.reset_index().drop_duplicates(subset=['證券代號', '日期'], keep='last').set_index(['證券代號', '日期']).sort_index()
  final_df.to_sql('daily_prices', connection, if_exists='replace')
  connection.close()

  excel_file = os.path.join('data', 'excel_files', 'daily_prices.xlsx')
  os.makedirs(os.path.dirname(excel_file), exist_ok=True)
  final_df.to_excel(excel_file)

In [43]:
save_daily_prices(df)